# Importing libraries

In [1]:
import os
import random
import numpy as np
import pandas as pd
import re
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset
import pandas as pd
from huggingface_hub import login
import sentencepiece
import accelerate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AlbertTokenizer
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support, balanced_accuracy_score
from sklearn.model_selection import train_test_split

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


# Loading the dataset

The dataset we are using is not publically available. You need to request the authors, to access this dataset

Name of Datset: HateDay, a global hate speech dataset  
Source: Tonneau et al. (2025), arXiv:2411.15462  
Available at: https://arxiv.org/abs/2411.15462

In [3]:
# Logging in Hugging Face
login()

# Loading dataset directly from HuggingFace Hub
dataset = load_dataset("manueltonneau/india-hate-speech-superset")

# Converting the HF dataset split into a pandas DataFrame
df = dataset["train"].to_pandas()

# Keeping only text + labels columns
df = df[["text", "labels"]].copy()

# Printing confirmation
print(df)
print("\n Datset loaded successfully")

                                                    text  labels
0      वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी ध...       0
1      #कांग्रेस के इस #कमीने की #करतूत को देखिए देश ...       0
2      पाकिस्तान को फेकना था फेका गया। जो हार कर भी द...       0
3      जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे व...       0
4      नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आ...       0
...                                                  ...     ...
14150  सोनू सूद से प्रेरणा लेकर आदिवासियों ने वो किया...       0
14151  RT @USER: उमर खालिद पर तो UAPA लगा दिया!\nकपिल...       1
14152                                  @USER पप्पू कमीना       0
14153  मदर टेरेसा ने अपना पूरा जीवन पीड़ितों की सेवा ...       0
14154  आईपीएल में कोरोना का डर: रैना के हटने के बाद ह...       0

[14155 rows x 2 columns]

 Datset loaded successfully


In [4]:
# Print available splits
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'source', 'dataset', 'nb_annotators'],
        num_rows: 14155
    })
})


In [5]:
# Basic overview of the dataset
print("Initial dataframe shape:", df.shape)
print("\nDataFrame info:")
print(df.info())

print("\nLabel distribution (raw):")
print(df["labels"].value_counts(dropna=False))

Initial dataframe shape: (14155, 2)

DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14155 entries, 0 to 14154
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14155 non-null  object
 1   labels  14155 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 221.3+ KB
None

Label distribution (raw):
labels
0    12281
1     1874
Name: count, dtype: int64


# Cleaning the dataset

In [6]:
# Defining a function to clean the dataset
def clean_hindi_text(text: str) -> str:
    """
    Very light cleaning of a Hindi / code-mixed string.
    - Lowercases Latin characters (does NOT touch Devanagari).
    - Removes URLs.
    - Removes @usernames.
    - Strips leading '#' but keeps the hashtag word (for both Latin & Hindi).
    - Normalises multiple spaces.
    """
    if not isinstance(text, str):
        text = str(text)
    
    # Removing URLs
    text = re.sub(r"http\S+|www\.\S+", " ", text)
    
    # Removing @usernames (any non-space after @)
    text = re.sub(r"@[^\s]+", " ", text)
    
    # Stripping '#' but keeping the following token (Latin or Hindi)
    # e.g. "#कांग्रेस" becomes "कांग्रेस", "#Modi" becomes "Modi" after cleaning
    text = re.sub(r"#(\S+)", r"\1", text)
    
    # lowercase Latin letters only
    text = "".join(
        ch.lower() if "A" <= ch <= "Z" or "a" <= ch <= "z" else ch
        for ch in text
    )
    
    # Collapsing multiple whitespace
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [7]:
# Applying the cleaning fucntion and creating a new column, "cleaned_text"
df["cleaned_text"] = df["text"].apply(clean_hindi_text)

# Checking rows after cleaning
df[["text", "cleaned_text", "labels"]].head(20)

,text,cleaned_text,labels
0,"वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी ध...","वक्त, इन्सान और इंग्लैंड का मौसम आपको कभी भी ध...",0
1,#कांग्रेस के इस #कमीने की #करतूत को देखिए देश ...,कांग्रेस के इस कमीने की करतूत को देखिए देश की ...,0
2,पाकिस्तान को फेकना था फेका गया। जो हार कर भी द...,पाकिस्तान को फेकना था फेका गया। जो हार कर भी द...,0
3,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे व...,जो शब्द तूम आज किसी और औरत के लिए यूज कर रहे व...,0
4,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आ...,नेता जी हम समाजवादी सिपाही हमेशा आपके साथ है आ...,0
5,@USER @USER @USER @USER @USER #AayegaTohModiHi...,aayegatohmodihi bjp4india bjp कन्हैया_कुमार ने...,0
6,#कुत्ते भी हो रहे हैं #किडनी और #डायबटीज जैसी ...,कुत्ते भी हो रहे हैं किडनी और डायबटीज जैसी बीम...,0
7,VIVAHIT BaitION KO BHI Patrick KIRSI BHOOMI MA...,vivahit baition ko bhi patrick kirsi bhoomi ma...,1
8,10 एजेंसियों को किसी भी कंप्यूटर की निगरानी और...,10 एजेंसियों को किसी भी कंप्यूटर की निगरानी और...,1
9,कश्मीर मे सेना पर पत्थर मारने वाले और पत्थरबाज...,कश्मीर मे सेना पर पत्थर मारने वाले और पत्थरबाज...,0


In [8]:
df.shape

(14155, 3)

In [9]:
# Removing duplicate tweets based on uncleaned datsaet column, "text"
# Before
rows_before = len(df)

# Boolean mask: True for rows that are duplicates of an earlier row 
dup_mask_text = df.duplicated(subset=["text"])
n_dups_text = dup_mask_text.sum()

print(f"Rows BEFORE dropping duplicates:     {rows_before}")
print(f"Exact duplicates (by 'text'):        {n_dups_text}")

# Dropping only those duplicates (keep the first occurrence)
df = df[~dup_mask_text].reset_index(drop=True)

rows_after = len(df)
print(f"Rows AFTER dropping duplicates:      {rows_after}")
print(f"Rows removed:                        {rows_before - rows_after}")

Rows BEFORE dropping duplicates:     14155
Exact duplicates (by 'text'):        28
Rows AFTER dropping duplicates:      14127
Rows removed:                        28


In [10]:
# Basic sanity checks
print("\nFinal dataset summary:")
print(f"Shape: {df.shape}")
print("\nLabel distribution (labels):")
print(df["labels"].value_counts())

print("\nLabel distribution by cleaned_text length:")
df["cleaned_len"] = df["cleaned_text"].str.len()
print(df["cleaned_len"].describe())



Final dataset summary:
Shape: (14127, 3)

Label distribution (labels):
labels
0    12257
1     1870
Name: count, dtype: int64

Label distribution by cleaned_text length:
count    14127.000000
mean       147.298790
std         83.962405
min          0.000000
25%         89.000000
50%        126.000000
75%        201.000000
max       2281.000000
Name: cleaned_len, dtype: float64


# Saving the datset

In [11]:
# Keeping both original 'text'
# And 'cleaned_text' as the actual model input.
final_df = df[["text", "cleaned_text", "labels"]].copy()

# Saving to CSV in current directory
output_csv = "hindi_hatespeech_cleaned.csv"
final_df.to_csv(output_csv, index=False, encoding="utf-8")

print(f"\n Saved cleaned dataset for model training to: {output_csv}")


 Saved cleaned dataset for model training to: hindi_hatespeech_cleaned.csv
